# RE RE RE FEATURE ENGINEERING OF DATA - applying transform

In [2]:
import numpy as np
import pandas as panda
from dateutil import relativedelta
import datetime

In [2]:
train_data_path = 'all_new/train.csv'
test_data_path = 'all_new/test.csv'
new_transaction_path = 'all_new/new_merchant_transactions.csv'
historical_transaction_path = 'all_new/historical_transactions.csv'
new_treated_transaction_path = 'all_new/treated_new_transaction.csv'
treated_historical_transaction_path = 'all_new/treated_historical_transactions.csv'



In [3]:
## deres nothing else we can do now...with this data
## lets look at new transactions first , since it is a smaller data sets

new_transaction_data = panda.read_csv(new_transaction_path)
new_transaction_data_types = new_transaction_data.dtypes
new_transaction_column_types = {}
for index, column_name in enumerate(new_transaction_data.columns.tolist()):
    new_transaction_column_types[column_name] = new_transaction_data_types[index]
del new_transaction_data
new_transaction_data = panda.read_csv(new_transaction_path, dtype = new_transaction_column_types)
new_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [4]:
## lets deal with the easy categorical data first: authorized_flag, category_1,category_2,category_3
new_transaction_data.authorized_flag.isnull().sum()
new_transaction_data['authorized_flag'] = new_transaction_data.authorized_flag.map({'Y':1,'N':0})

In [5]:
new_transaction_data.category_1.isnull().sum()
new_transaction_data.category_1.value_counts()
new_transaction_data['category_1'] = new_transaction_data.category_1.map({'Y':1, 'N':0})

In [6]:
new_transaction_data.category_2.isnull().sum()
##quite a few null values in category 2
new_transaction_data.category_2.value_counts()
most_appearing = new_transaction_data.category_2.value_counts().idxmax()
new_transaction_data.category_2.fillna(value= most_appearing, inplace=True)
new_transaction_data.category_2.value_counts()

1.0    1169987
3.0     289525
5.0     259266
4.0     178590
2.0      65663
Name: category_2, dtype: int64

In [7]:
new_transaction_data['enc_category_2_1'] = (new_transaction_data.category_2==1) + 0
new_transaction_data['enc_category_2_2'] = (new_transaction_data.category_2==2) + 0
new_transaction_data['enc_category_2_3'] = (new_transaction_data.category_2==3) + 0
new_transaction_data['enc_category_2_4'] = (new_transaction_data.category_2==4) + 0
new_transaction_data['enc_category_2_5'] = (new_transaction_data.category_2==5) + 0

new_transaction_data.drop(columns = ['category_2'], inplace=True)
new_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,enc_category_2_1,enc_category_2_2,enc_category_2_3,enc_category_2_4,enc_category_2_5
0,1,C_ID_415bb3a509,107,0,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,1,0,0,0,0
1,1,C_ID_415bb3a509,140,0,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,1,0,0,0,0
2,1,C_ID_415bb3a509,330,0,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,9,14,1,0,0,0,0
3,1,C_ID_415bb3a509,-1,1,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,-1,8,1,0,0,0,0
4,1,C_ID_ef55cf8d4b,-1,1,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,-1,29,1,0,0,0,0


In [9]:
new_transaction_data.category_3.isnull().sum()
## again quite a few null values for category
most_appearing = new_transaction_data.category_3.value_counts().idxmax()
new_transaction_data.category_3.fillna(value = most_appearing, inplace = True)
new_transaction_data.category_3.value_counts(),new_transaction_data.category_3.isnull().sum()

(A    978166
 B    836178
 C    148687
 Name: category_3, dtype: int64, 0)

In [10]:
new_transaction_data['enc_category_3_A'] = (new_transaction_data.category_3=='A') +0
new_transaction_data['enc_category_3_B'] = (new_transaction_data.category_3=='B') +0
new_transaction_data['enc_category_3_C'] = (new_transaction_data.category_3=='C') +0
new_transaction_data.drop(columns = ['category_3'], inplace = True)

In [10]:
new_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,enc_category_2_1,enc_category_2_2,enc_category_2_3,enc_category_2_4,enc_category_2_5,enc_category_3_A,enc_category_3_B,enc_category_3_C
0,1,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,1,0,0,0,0,0,1,0
1,1,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,1,0,0,0,0,0,1,0
2,1,C_ID_415bb3a509,330,0,1,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,9,14,1,0,0,0,0,0,1,0
3,1,C_ID_415bb3a509,-1,1,1,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,-1,8,1,0,0,0,0,0,1,0
4,1,C_ID_ef55cf8d4b,-1,1,1,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,-1,29,1,0,0,0,0,0,1,0


In [11]:
new_transaction_data['enc_purchase_date'] = panda.DatetimeIndex(new_transaction_data.purchase_date).astype(np.int64)*1e-9
new_transaction_data.drop(columns = ['purchase_date'], inplace = True)


In [12]:
new_transaction_data.city_id.isnull().sum()
new_transaction_data.city_id.value_counts()
most_appearing = new_transaction_data.city_id.value_counts().idxmax()
new_transaction_data.city_id.replace({-1:most_appearing}, inplace = True)
new_transaction_data.city_id.isnull().sum()

0

In [13]:
new_transaction_data.state_id.isnull().sum()
new_transaction_data.state_id.value_counts()
most_appearing = new_transaction_data.state_id.value_counts().idxmax()
new_transaction_data.state_id.replace({-1:most_appearing}, inplace = True)
new_transaction_data.state_id.value_counts()


9     844891
15    158304
16    139765
5     120158
19     87980
20     78953
22     65300
4      63532
21     60155
7      49342
11     46905
13     39432
3      38908
2      27814
12     27027
18     22139
1      17908
10     17273
23     13069
24     12825
8      12218
14     10326
17      8450
6        357
Name: state_id, dtype: int64

In [14]:
new_transaction_data.subsector_id.isnull().sum()
new_transaction_data.subsector_id.value_counts()
most_appearing = new_transaction_data.subsector_id.value_counts().idxmax()
new_transaction_data.subsector_id.replace({-1:most_appearing}, inplace = True)
new_transaction_data.subsector_id.value_counts()

37    340073
19    200528
27    194977
34    183700
33    172585
29    138039
16    119297
21     74176
1      66913
7      54145
18     42079
20     38433
36     36621
30     31443
15     29772
25     23805
17     23233
39     21487
12     21351
32     16673
22     16460
2      15851
31     15000
8      14979
38     13057
41     12848
10      9849
9       7590
3       6237
35      5233
4       4302
13      2419
26      2221
24      1907
14      1488
23      1423
5       1316
40       953
11       560
28         8
Name: subsector_id, dtype: int64

In [15]:
## we will take the 
## sum,mean,max of purchase amount, 
## sum,mean,max of month lag by card_id
## mean of authorized_flag,
## num of authorized ==1
## num of authorized ==0
## sum, mean, max of installments
## mean of category_1
## sum of enc_category_2* and enc_category_3*

def count_authorized_one(x):
    return x.eq(1).sum()
def count_authorized_zero(x):
    return x.eq(0).sum()
def count_category_1_one(x):
    return x.eq(1).sum()
def count_category_1_zero(x):
    return x.eq(0).sum()

def count_of_unique_cities(x):
    return x.nunique()

def count_of_unique_subsector_id(x):
    return x.nunique()

def count_of_unique_state_id(x):
    return x.nunique()


In [16]:

aggregate = {
    
'purchase_amount' : {'new_purchase_amount_sum':'sum','new_purchase_amount_mean': 'mean','new_purchase_amount_max':'max'},
'month_lag' : {'new_month_lag_sum':'sum', 'new_month_lag_mean':'mean','new_month_lag_max':'max'} ,
'enc_category_2_1':{'new_category_2_1_sum':'sum'},
'enc_category_2_2':{'new_category_2_2_sum':'sum'},
'enc_category_2_3':{'new_category_2_3_sum':'sum'},
'enc_category_2_4':{'new_category_2_4_sum':'sum'},
'enc_category_2_5':{'new_category_2_5_sum':'sum'},    
'enc_category_3_A':{'new_category_3_A_sum':'sum'},
'enc_category_3_B':{'new_category_2_B_sum':'sum'},
'enc_category_3_C':{'new_category_3_C_sum':'sum'},
'installments': {'new_installments_sum':'sum','new_installments_mean':'mean','new_installments_max':'max'},
'authorized_flag': {'new_authorized_flag_mean':'mean', 'new_authorized_flag_count_authorized_one':count_authorized_one, 'new_authorized_flag_count_authorized_zero':count_authorized_zero},
'category_1' :{'new_category_1_mean':'mean', 'new_category_1_count_category_1_one':count_category_1_one, 'new_category_1_count_category_1_zero':count_category_1_zero},
'enc_purchase_date' : {'new_purchase_date_mean':'mean','new_purchase_date_min':'min','new_purchase_date_max':'max'},
'city_id' : {'new_city_id_nunique':count_of_unique_cities},
'state_id': {'new_state_id_nunique':count_of_unique_state_id},
'subsector_id' : {'new_subsector_id_nunique':count_of_unique_subsector_id}
    
}



In [17]:
group_by = new_transaction_data.groupby(['card_id'])

In [18]:
len(new_transaction_data)

1963031

In [19]:
for key in aggregate.keys():
    operations = aggregate.get(key)
    for key_2 in operations.keys():
        new_transaction_data[key_2] = group_by[key].transform(operations.get(key_2))
#         print(operations.get(key_2))

len(new_transaction_data), new_transaction_data.isnull().sum()

(1963031, authorized_flag                                  0
 card_id                                          0
 city_id                                          0
 category_1                                       0
 installments                                     0
 merchant_category_id                             0
 merchant_id                                  26216
 month_lag                                        0
 purchase_amount                                  0
 state_id                                         0
 subsector_id                                     0
 enc_category_2_1                                 0
 enc_category_2_2                                 0
 enc_category_2_3                                 0
 enc_category_2_4                                 0
 enc_category_2_5                                 0
 enc_category_3_A                                 0
 enc_category_3_B                                 0
 enc_category_3_C                                 0
 en

In [37]:

new_transaction_data.head()
# t = group_by.agg(aggregate)

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,state_id,...,new_authorized_flag_count_authorized_zero,new_category_1_mean,new_category_1_count_category_1_one,new_category_1_count_category_1_zero,new_purchase_date_mean,new_purchase_date_min,new_purchase_date_max,new_city_id_nunique,new_state_id_nunique,new_subsector_id_nunique
0,1,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.557574,9,...,0,0.250000,1,3,1.521858e+09,1.520416e+09,1.524752e+09,4,1,3
1,1,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.569580,9,...,0,0.250000,1,3,1.521858e+09,1.520416e+09,1.524752e+09,4,1,3
2,1,C_ID_415bb3a509,330,0,1,507,M_ID_ad5237ef6b,2,-0.551037,9,...,0,0.250000,1,3,1.521858e+09,1.520416e+09,1.524752e+09,4,1,3
3,1,C_ID_415bb3a509,69,1,1,661,M_ID_9e84cda3b1,1,-0.671925,9,...,0,0.250000,1,3,1.521858e+09,1.520416e+09,1.524752e+09,4,1,3
4,1,C_ID_ef55cf8d4b,69,1,1,166,M_ID_3c86fa3831,1,-0.659904,9,...,0,0.181818,4,18,1.522046e+09,1.519914e+09,1.524409e+09,2,1,12


In [20]:
#i dont need merchant id, so im drping it
new_transaction_data.drop(columns =['merchant_id'], inplace = True)

In [ ]:
new_transaction_data.to_csv('all_new/treated_new_transaction_2.csv', index = False)

In [3]:
## deres nothing else we can do now...with this data
## lets look at new transactions first , since it is a smaller data sets

historical_transaction_data = panda.read_csv('all_new/historical_transactions.csv')
new_transaction_data_types = historical_transaction_data.dtypes
new_transaction_column_types = {}
for index, column_name in enumerate(historical_transaction_data.columns.tolist()):
    new_transaction_column_types[column_name] = new_transaction_data_types[index]
del historical_transaction_data
historical_transaction_data = panda.read_csv('all_new/historical_transactions.csv', dtype = new_transaction_column_types)
historical_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [4]:
## lets deal with the easy categorical data first: authorized_flag, category_1,category_2,category_3
historical_transaction_data.authorized_flag.isnull().sum()
historical_transaction_data['authorized_flag'] = historical_transaction_data.authorized_flag.map({'Y':1,'N':0})

In [5]:
historical_transaction_data.category_1.isnull().sum()
historical_transaction_data.category_1.value_counts()
historical_transaction_data['category_1'] = historical_transaction_data.category_1.map({'Y':1, 'N':0})

In [6]:
historical_transaction_data.category_2.isnull().sum()
##quite a few null values in category 2
historical_transaction_data.category_2.value_counts()
most_appearing = historical_transaction_data.category_2.value_counts().idxmax()
historical_transaction_data.category_2.fillna(value= most_appearing, inplace=True)
# historical_transaction_data.category_2.value_counts()

In [7]:
historical_transaction_data['enc_category_2_1'] = (historical_transaction_data.category_2==1) + 0
historical_transaction_data['enc_category_2_2'] = (historical_transaction_data.category_2==2) + 0
historical_transaction_data['enc_category_2_3'] = (historical_transaction_data.category_2==3) + 0
historical_transaction_data['enc_category_2_4'] = (historical_transaction_data.category_2==4) + 0
historical_transaction_data['enc_category_2_5'] = (historical_transaction_data.category_2==5) + 0

historical_transaction_data.drop(columns = ['category_2'], inplace=True)
historical_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,enc_category_2_1,enc_category_2_2,enc_category_2_3,enc_category_2_4,enc_category_2_5
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,1,0,0,0,0
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,1,0,0,0,0
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,1,0,0,0,0
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,16,34,1,0,0,0,0
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,16,37,1,0,0,0,0


In [8]:
historical_transaction_data.category_3.isnull().sum()
## again quite a few null values for category
most_appearing = historical_transaction_data.category_3.value_counts().idxmax()
historical_transaction_data.category_3.fillna(value = most_appearing, inplace = True)
# historical_transaction_data.category_3.value_counts()
historical_transaction_data.category_3.isnull().sum()

0

In [9]:
historical_transaction_data['enc_category_3_A'] = (historical_transaction_data.category_3=='A') +0
historical_transaction_data['enc_category_3_B'] = (historical_transaction_data.category_3=='B') +0
historical_transaction_data['enc_category_3_C'] = (historical_transaction_data.category_3=='C') +0
historical_transaction_data.drop(columns = ['category_3'], inplace = True)

In [53]:
historical_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,enc_category_2_1,enc_category_2_2,enc_category_2_3,enc_category_2_4,enc_category_2_5,enc_category_3_A,enc_category_3_B,enc_category_3_C
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,1,0,0,0,0,1,0,0
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,1,0,0,0,0,1,0,0
2,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,1,0,0,0,0,1,0,0
3,1,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,16,34,1,0,0,0,0,1,0,0
4,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,16,37,1,0,0,0,0,1,0,0


In [10]:
historical_transaction_data['enc_purchase_date'] = panda.DatetimeIndex(historical_transaction_data.purchase_date).astype(np.int64)*1e-9
historical_transaction_data.drop(columns = ['purchase_date'], inplace = True)


In [11]:
historical_transaction_data.city_id.isnull().sum()
historical_transaction_data.city_id.value_counts()
most_appearing = historical_transaction_data.city_id.value_counts().idxmax()
historical_transaction_data.city_id.replace({-1:most_appearing}, inplace = True)
historical_transaction_data.city_id.isnull().sum()

0

In [12]:
historical_transaction_data.state_id.isnull().sum()
historical_transaction_data.state_id.value_counts()
most_appearing = historical_transaction_data.state_id.value_counts().idxmax()
historical_transaction_data.state_id.replace({-1:most_appearing}, inplace = True)
# historical_transaction_data.state_id.value_counts()


In [13]:
historical_transaction_data.subsector_id.isnull().sum()
historical_transaction_data.subsector_id.value_counts()
most_appearing = historical_transaction_data.subsector_id.value_counts().idxmax()
historical_transaction_data.subsector_id.replace({-1:most_appearing}, inplace = True)
# historical_transaction_data.subsector_id.value_counts()

In [14]:
## we will take the 
## sum,mean,max of purchase amount, 
## sum,mean,max of month lag by card_id
## mean of authorized_flag,
## num of authorized ==1
## num of authorized ==0
## sum, mean, max of installments
## mean of category_1
## sum of enc_category_2* and enc_category_3*

def count_authorized_one(x):
    return x.eq(1).sum()
def count_authorized_zero(x):
    return x.eq(0).sum()
def count_category_1_one(x):
    return x.eq(1).sum()
def count_category_1_zero(x):
    return x.eq(0).sum()

def count_of_unique_cities(x):
    return x.nunique()

def count_of_unique_subsector_id(x):
    return x.nunique()

def count_of_unique_state_id(x):
    return x.nunique()


In [15]:

aggregate = {
    
'purchase_amount' : {'old_purchase_amount_sum':'sum','old_purchase_amount_mean': 'mean','old_purchase_amount_max':'max'},
'month_lag' : {'old_month_lag_sum':'sum', 'old_month_lag_mean':'mean','old_month_lag_max':'max'} ,
'enc_category_2_1':{'old_category_2_1_sum':'sum'},
'enc_category_2_2':{'old_category_2_2_sum':'sum'},
'enc_category_2_3':{'old_category_2_3_sum':'sum'},
'enc_category_2_4':{'old_category_2_4_sum':'sum'},
'enc_category_2_5':{'old_category_2_5_sum':'sum'},    
'enc_category_3_A':{'old_category_3_A_sum':'sum'},
'enc_category_3_B':{'old_category_3_B_sum':'sum'},
'enc_category_3_C':{'old_category_3_C_sum':'sum'},
'installments': {'old_installments_sum':'sum','old_installments_mean':'mean','old_installments_max':'max'},
'authorized_flag': {'old_authorized_flag_mean':'mean', 'old_authorized_flag_count_authorized_one':count_authorized_one, 'old_authorized_flag_count_authorized_zero':count_authorized_zero},
'category_1' :{'old_category_1_mean':'mean', 'old_category_1_count_category_1_one':count_category_1_one, 'old_category_1_count_category_1_zero':count_category_1_zero},
'enc_purchase_date' : {'old_purchase_date_mean':'mean','old_purchase_date_min':'min','old_purchase_date_max':'max'},
'city_id' : {'old_city_id_nunique':count_of_unique_cities},
'state_id': {'old_state_id_nunique':count_of_unique_state_id},
'subsector_id' : {'old_subsector_id_nunique':count_of_unique_subsector_id}
    
}



In [16]:
group_by = historical_transaction_data.groupby(['card_id'])

In [17]:
len(historical_transaction_data)

29112361

In [17]:
for key in aggregate.keys():
    operations = aggregate.get(key)
    for key_2 in operations.keys():
        historical_transaction_data[key_2] = group_by[key].transform(operations.get(key_2))
#         print(operations.get(key_2))

# len(historical_tra?nsaction_data), historical_transaction_data.isnull().sum()

MemoryError: 

In [18]:
import gc
gc.collect()

179

In [19]:
len(historical_transaction_data)
# historical_transaction_data.head()
# t = group_by.agg(aggregate)

29112361

In [20]:
#i dont need merchant id, so im drping it
historical_transaction_data.drop(columns =['merchant_id'], inplace = True)

MemoryError: 

In [21]:
historical_transaction_data.columns.tolist()

['authorized_flag',
 'card_id',
 'city_id',
 'category_1',
 'installments',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'state_id',
 'subsector_id',
 'enc_category_2_1',
 'enc_category_2_2',
 'enc_category_2_3',
 'enc_category_2_4',
 'enc_category_2_5',
 'enc_category_3_A',
 'enc_category_3_B',
 'enc_category_3_C',
 'enc_purchase_date',
 'old_purchase_amount_sum',
 'old_purchase_amount_mean',
 'old_purchase_amount_max',
 'old_month_lag_sum',
 'old_month_lag_mean',
 'old_month_lag_max',
 'old_category_2_1_sum',
 'old_category_2_2_sum',
 'old_category_2_3_sum',
 'old_category_2_4_sum',
 'old_category_2_5_sum',
 'old_category_3_A_sum',
 'old_category_3_B_sum',
 'old_category_3_C_sum',
 'old_installments_sum',
 'old_installments_mean',
 'old_installments_max',
 'old_authorized_flag_mean',
 'old_authorized_flag_count_authorized_one',
 'old_authorized_flag_count_authorized_zero',
 'old_category_1_mean',
 'old_category_1_count_category_1_one',
 'old_category_1_

In [22]:
historical_transaction_data.to_csv('all_new/treated_historical_transaction_2.csv', index = False)